<a href="https://colab.research.google.com/github/Ayush-mishra-0-0/ML/blob/main/ayushKumarMishra_12240340_aiml_assign2_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ques-1

In [4]:
!pip install roboflow  # Install the roboflow library

# Import required libraries
import librosa
import torch
from transformers import pipeline
!pip install jiwer # Install the jiwer library
from jiwer import wer
from roboflow import Roboflow # Now you should be able to import roboflow
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
import numpy as np
import os
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 41.6 MB/s eta 0:00:00


In [6]:
# Part 1: Whisper Model for Audio Transcription
# Step 1: Load Whisper Model
whisper_model = pipeline("automatic-speech-recognition", model="openai/whisper-base")

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
# Step 2: Load and Transcribe Audio Samples
audio_files = ["/content/ai1.unknown", "/content/ai3.unknown", "/content/ai4.unknown", "/content/ai5.unknown", "/content/ai6.unknown"]  # Replace with your own file paths
ground_truths = [
    "Data Science is crucial for managing, processing, and analyzing vast amounts of data (often referred to as Big Data) to discover patterns, trends, and relationships that can inform decision-making..",
    "Data Science is an interdisciplinary field that involves extracting insights and knowledge from structured and unstructured data using scientific methods, processes, algorithms, and systems.",
    "AI is revolutionizing industries by automating tasks that were previously done manually.",
    "A key component of Data Science is the visualization of data. Tools like Matplotlib, Seaborn, and Tableau help in presenting data in a visual context, making it easier to identify patterns and insights.",
    "Machine Learning is A crucial subset of AI, which focuses on creating algorithms that allow computers to learn from and make predictions based on data. It is heavily used in data science for predictive modeling."
]

transcriptions = []
error_rates = []

for audio_file, ground_truth in zip(audio_files, ground_truths):
    # Load and preprocess audio
    audio, sr = librosa.load(audio_file, sr=16000)
    transcription = whisper_model(audio)["text"]
    transcriptions.append(transcription)
    error_rate = wer(ground_truth, transcription)
    error_rates.append(error_rate)
    print(f"Transcription: {transcription}")
    print(f"WER for {audio_file}: {error_rate}")

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription:  Data Science is a crucial for managing, processing and analyzing vast amount of data often referred to as big data to discover patterns, trends and relationships that can inform decision making.
WER for /content/ai1.unknown: 0.3103448275862069
Transcription:  Data Science is an interdisciplinary field that involves extracting insights and knowledge from structured and unstructured data using scientific methods, processes, algorithms and systems.
WER for /content/ai3.unknown: 0.041666666666666664
Transcription:  AI is revolutionizing industries by automaking tasks that are previously done manually.
WER for /content/ai4.unknown: 0.16666666666666666
Transcription:  A key component of data science is the visualization of data. Tools like MAT, plot, IBC, bone, table U help in presenting data in a visual context making it easier to identify patterns and insights.
WER for /content/ai5.unknown: 0.2727272727272727
Transcription:  Machine Learning is a crucial subset of AI which 

# QUES-2

In [ ]:
! pip install torch torchvision timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.6 MB/s eta 0:00:00


In [ ]:
import os
import zipfile
import requests

url = "https://public.roboflow.com/ds/on5dKPftN9?key=vjl0Jd4vmE"
dataset_path = "roboflow_dataset.zip"


response = requests.get(url)
with open(dataset_path, "wb") as file:
    file.write(response.content)

with zipfile.ZipFile(dataset_path, "r") as zip_ref:
    zip_ref.extractall("roboflow_dataset")

os.remove(dataset_path)


In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(root='roboflow_dataset/train', transform=transform)
val_dataset = datasets.ImageFolder(root='roboflow_dataset/valid', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import timm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model('deit_base_patch16_224', pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_labels = len(train_dataset.classes)
model.head = nn.Sequential(
    nn.Linear(model.head.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_labels)
)

model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.head.parameters(), lr=1e-3)


In [ ]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")


Epoch 1/10, Loss: 0.1982
Epoch 2/10, Loss: 0.1016
Epoch 3/10, Loss: 0.0738
Epoch 4/10, Loss: 0.0561
Epoch 5/10, Loss: 0.0352
Epoch 6/10, Loss: 0.0235
Epoch 7/10, Loss: 0.0197
Epoch 8/10, Loss: 0.0173
Epoch 9/10, Loss: 0.0113
Epoch 10/10, Loss: 0.0117


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Validation Accuracy: 95.60%
